In [5]:
#!pip3 install pickle5

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

import sys

PATH = '/content/drive/MyDrive/Colab Notebooks/CS598_DLH_Paper211'
sys.path.append(PATH)


Mounted at /content/drive


In [2]:
import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText
# import glove
# from glove import Corpus

import collections
import gc

import warnings
warnings.filterwarnings('ignore')


In [5]:
!pip3 install pickle5
import pickle5 as pickle
with open(PATH + "/data/ner_df.p", "rb") as fh:
  new_notes = pickle.load(fh)

In [7]:
# new_notes = pd.read_pickle(PATH + "/data/ner_df.p")  # med7

In [8]:
new_notes

,SUBJECT_ID,HADM_ID_y,CHARTTIME,TEXT,preprocessed_text,ner
16,27866,109679,2143-04-09 03:22:00,Ms. [**Known lastname 1170**] is an 83 yo F w/...,"[ms. is an 83 yo f w/pmhx sx for cll , hyperte...","[[(6u, DOSAGE), (prbc, DRUG)], [(prbcs, DRUG)]..."
17,27866,109679,2143-04-09 03:22:00,Ms. [**Known lastname 1170**] is an 83 yo F w/...,"[ms. is an 83 yo f w/pmhx sx for cll , hyperte...","[[(6u, DOSAGE), (prbc, DRUG)], [(prbcs, DRUG)]..."
23,29075,179159,2116-02-06 00:25:00,Chief Complaint: GI bleeding\n HPI:\n 67M...,[chief complaint : gi bleeding hpi : 67m w/ h/...,"[[(2u, DOSAGE), (prbc, DRUG), (6u, DOSAGE), (p..."
24,29075,179159,2116-02-06 00:25:00,Chief Complaint: GI bleeding\n HPI:\n 67M...,[chief complaint : gi bleeding hpi : 67m w/ h/...,"[[(2u, DOSAGE), (prbc, DRUG), (6u, DOSAGE), (p..."
25,29075,179159,2116-02-06 03:50:00,Chief Complaint: GI Bleed\n I saw and exami...,[chief complaint : gi bleed i saw and examined...,"[[(6 units, DOSAGE), (plts, DRUG), (2 units, D..."
...,...,...,...,...,...,...
745917,46335,114333,2101-07-18 09:22:00,Chief Complaint: Hypoxia\n 24 Hour Events:\n...,[chief complaint : hypoxia 24 hour events : - ...,"[[(antibiotics, DRUG), (infusions, ROUTE), (he..."
745926,41441,149099,2153-02-03 07:08:00,Chief Complaint:\n 24 Hour Events:\n -EGD ...,[chief complaint : 24 hour events : -egd w/ 4 ...,"[[(octreotide, DRUG), (gtt, ROUTE), (nadolol, ..."
745927,76261,146784,2145-03-06 07:45:00,TITLE:\n Chief Complaint: 73 year old man wi...,[title : chief complaint : 73 year old man wit...,"[[(warfarin, DRUG)], [(wgt, DRUG)]]"
745932,40387,161638,2160-03-08 06:15:00,Chief Complaint:\n 24 Hour Events:\n NASAL S...,[chief complaint : 24 hour events : nasal swab...,"[[(antibiotics, DRUG), (infusions, ROUTE), (do..."


In [9]:

w2vec = Word2Vec.load(PATH + "/embeddings/word2vec.model")
w2vec = w2vec.wv
fasttext = FastText.load(PATH + "/embeddings/FastText/fasttext.model")
fasttext = fasttext.wv
new_notes.columns

Index(['SUBJECT_ID', 'HADM_ID_y', 'CHARTTIME', 'TEXT', 'preprocessed_text',
       'ner'],
      dtype='object')

In [10]:
null_index_list = []
for i in new_notes.itertuples():
    if len(i.ner) == 0:
        null_index_list.append(i.Index)
new_notes.drop(null_index_list, inplace=True)


In [11]:
def mean(a):
    return sum(a) / len(a)

In [12]:
med7_ner_data = {}

for ii in new_notes.itertuples():

    p_id = ii.SUBJECT_ID
    ind = ii.Index

    try:
        new_ner = new_notes.loc[ind].ner
    except:
        new_ner = []

    unique = set()
    new_temp = []

    for j in new_ner:
        for k in j:

            unique.add(k[0])
            new_temp.append(k)

    if p_id in med7_ner_data:
        for i in new_temp:
            med7_ner_data[p_id].append(i)
    else:
        med7_ner_data[p_id] = new_temp


In [14]:
med7_ner_data[1106]
new_ner
# len(med7_ner_data[1106])

ner_count_dict={}

for ner_item in med7_ner_data.values():
    for ner in ner_item:
        if ner[1] not in ner_count_dict.keys():
            ner_count_dict[ner[1]] = 1
        ner_count_dict[ner[1]] += 1

ner_count_dict

{'DOSAGE': 84131,
 'DRUG': 684197,
 'DURATION': 6304,
 'FORM': 19583,
 'FREQUENCY': 131608,
 'ROUTE': 215924,
 'STRENGTH': 186852}

In [ ]:
pd.to_pickle(med7_ner_data, PATH + "/data/new_ner_word_dict.pkl")



In [ ]:
# w2vec['at 9p , by 10:15- pt with runs']
# fasttext['q6h']


In [ ]:

data_types = [med7_ner_data]
data_names = ["new_ner"]

for data, names in zip(data_types, data_names):
    new_word2vec = {}
    print("w2vec starting..")
    for k,v in data.items():

        patient_temp = []
        for i in v:
            try:
                patient_temp.append(w2vec[i[0]])
            except:
                avg = []
                num = 0
                temp = []

                if len(i[0].split(" ")) > 1:
                    for each_word in i[0].split(" "):
                        try:
                            temp = w2vec[each_word]
                            avg.append(temp)
                            num += 1
                        except:
                            pass
                    if num == 0: continue
                    avg = np.asarray(avg)
                    t = np.asarray(list(map(np.mean, zip(*avg))))
                    patient_temp.append(t)
        if len(patient_temp) == 0: 
            continue
        new_word2vec[k] = patient_temp
    print("w2vec Done..")
    #############################################################################
    print("fasttext starting..")
        
    new_fasttextvec = {}
    for k,v in data.items():
        patient_temp = []
        for i in v:
            try:
                patient_temp.append(fasttext[i[0]])
            except:
                pass
        if len(patient_temp) == 0: 
          continue
        new_fasttextvec[k] = patient_temp
    print("fasttext Done..")
    #############################################################################    
        
    print("combined starting..")
    new_concatvec = {}

    for k,v in data.items():
        patient_temp = []
    #     if k != 6: continue
        for i in v:
            w2vec_temp = []
            try:
                w2vec_temp = w2vec[i[0]]
            except:
                avg = []
                num = 0
                temp = []

                if len(i[0].split(" ")) > 1:
                    for each_word in i[0].split(" "):
                        try:
                            temp = w2vec[each_word]
                            avg.append(temp)
                            num += 1
                        except:
                            pass
                    if avg:
                        avg = np.asarray(avg)
                        w2vec_temp = np.asarray(list(map(np.mean, zip(*avg))))
                    else: 
                        w2vec_temp = [0] * 100
                else:
                    w2vec_temp = [0] * 100

            try:
                fasttemp = fasttext[i[0]]
            except:
                fasttemp = [0] * 100

            appended = np.append(fasttemp, w2vec_temp, 0)

            patient_temp.append(appended)
        if len(patient_temp) == 0: continue
        new_concatvec[k] = patient_temp

    print(len(new_word2vec), len(new_fasttextvec), len(new_concatvec))
    pd.to_pickle(new_word2vec, PATH + "/data/"+names+"_word2vec_dict.pkl")
    pd.to_pickle(new_fasttextvec, PATH + "/data/"+names+"_fasttext_dict.pkl")
    pd.to_pickle(new_concatvec, PATH + "/data/"+names+"_combined_dict.pkl")

w2vec starting..
w2vec Done..
fasttext starting..
fasttext Done..
combined starting..
21731 21809 21837


In [ ]:
diff = set(new_fasttextvec.keys()).difference(set(new_word2vec))
print(diff)


{90628, 91652, 97803, 74260, 41494, 9751, 91672, 95770, 73770, 94256, 68663, 86589, 80464, 58452, 45657, 84063, 79461, 96361, 44652, 63113, 51361, 86694, 18601, 4268, 55987, 67256, 41163, 55503, 83151, 66775, 70871, 9449, 55017, 92907, 14581, 66295, 45816, 95995, 67323, 89854, 48910, 98574, 26387, 76058, 69407, 56100, 81194, 10028, 41783, 90430, 78143, 77128, 44369, 68435, 59231, 46439, 57199, 95603, 41844, 63354, 93565, 83838, 50561, 6541, 83856, 82339, 65956, 90539, 92610, 83398, 85968, 85457, 73684, 44511, 55267, 53228, 85997, 79858}


In [ ]:

for i in diff:
    del new_fasttextvec[i]
    del new_concatvec[i]
print(len(new_word2vec), len(new_fasttextvec), len(new_concatvec))


pd.to_pickle(new_word2vec, PATH + "/data/"+"new_ner"+"_word2vec_limited_dict.pkl")
pd.to_pickle(new_fasttextvec, PATH + "/data/"+"new_ner"+"_fasttext_limited_dict.pkl")
pd.to_pickle(new_concatvec, PATH + "/data/"+"new_ner"+"_combined_limited_dict.pkl")


21731 21731 21759


In [ ]:
len(new_fasttextvec)

21731